#Pràctica 2: Recomanadors basats en continguts

David Galera

Es demana fer un programa (notebook) que:

# a/ A partir de les dades, generi una matriu on les files siguin les pel·lícules i les columnes els tags. A la posició (p_i,t_j) ha d’haver el nombre de vegades que algun usuari ha aplicat el tag t_j a la pel·lícula pi.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

In [3]:
tags = pd.read_csv("/content/drive/MyDrive/Advanced ML/tags.csv")
print('tags shape:',tags.shape)
print(tags.columns)
tags.head()

tags shape: (3683, 4)
Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [4]:
movies = pd.read_csv("/content/drive/MyDrive/Advanced ML/movies.csv")
print('movies shape:',movies.shape)
print(movies.columns)
movies.head()

movies shape: (9742, 3)
Index(['movieId', 'title', 'genres'], dtype='object')


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Afegim la columna amb el títol de la pel·lícula:

In [5]:
df = pd.merge(tags[['movieId','tag']],movies[['movieId','title']],on=['movieId'],how='left')
df.head()

,movieId,tag,title
0,60756,funny,Step Brothers (2008)
1,60756,Highly quotable,Step Brothers (2008)
2,60756,will ferrell,Step Brothers (2008)
3,89774,Boxing story,Warrior (2011)
4,89774,MMA,Warrior (2011)


Imprimim els tags de la pel·lícula *Step Brothers (2008)* i les unitats a mode d'exemple:

In [6]:
df[df['title']=='Step Brothers (2008)']['tag'].value_counts()

funny              3
will ferrell       3
Highly quotable    1
comedy             1
Name: tag, dtype: int64

Calculem el número de pel·lícules diferents que contenen algún tag i el número de tags diferents: 

In [7]:
print('Número de pel·lícules amb tags: ',df.movieId.nunique())
print('Número de tags diferents: ',df.tag.nunique())

Número de pel·lícules amb tags:  1572
Número de tags diferents:  1589


Per tant, segons l'enunciat hem de construir una matriu de 1572 files i 1589 columnes:

In [8]:
pivot=df.pivot_table(index='title',columns='tag',aggfunc='count')
pivot.columns=pivot.columns.droplevel(level=0)
pivot.fillna(0,inplace=True)
print(pivot.shape)
pivot.head()

(1572, 1589)


tag,"""artsy""",06 Oscar Nominated Best Movie - Animation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001-like,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Tornem a imprimir els tags de la pel·lícula *Step Brothers (2008)* i comprovem que efectivament a la posició (pi,tj) hi ha el nombre de vegades que **la pel·lícula pi ha rebut el tag tj**

In [9]:
pivot.loc['Step Brothers (2008)'][pivot.loc['Step Brothers (2008)']>0].sort_values(ascending=False)

tag
funny              3.0
will ferrell       3.0
Highly quotable    1.0
comedy             1.0
Name: Step Brothers (2008), dtype: float64

#b/ A partir d’aquesta matriu ens permeti introduir el codi d’una pel·lícula i ens proporcioni una recomanació de les N pel·lícules més similars. Aquesta recomanació l’ha de poder fer fent servir tres funcions de similitud diferents:
- Similitud de Jaccard per a conjunts
- Similitud de Jaccard per a multiconjunts (bags)
- Similitud del cosinus

- Comencem calculant la similitud de **Jaccard per a conjunts**, per fer-ho treballarem amb un dataframe binari:

In [26]:
binary=pivot.applymap(lambda x: x if x==0 else 1)

In [27]:
binary.head()

tag,"""artsy""",06 Oscar Nominated Best Movie - Animation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001-like,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane (2016),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Calculem la distància de Jaccard entre tots els parells de pel·lícules i posteriorment la matriu de similituds:

In [28]:
jaccard_distances=pdist(binary.values, metric='jaccard')
square_jaccard_distances=squareform(jaccard_distances)
square_jaccard_distances
square_jaccard_similarities=1-square_jaccard_distances
square_jaccard_similarities

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

Creem un dataframe amb la similitud de Jaccard entre tots els parells de pel·lícules:

In [29]:
df_jaccard=pd.DataFrame(square_jaccard_similarities, index=pivot.index,columns=pivot.index)

In [30]:
df_jaccard.head()

title,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),"11'09""01 - September 11 (2002)",12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),...,Young Frankenstein (1974),Z (1969),Zack and Miri Make a Porno (2008),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zombieland (2009),Zoolander (2001),Zulu (1964),eXistenZ (1999)
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane (2016),0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You (1999),0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (1996),0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Cerquem una recomanació de 10 pel·lícules similars a *Godfather: Part II, The (1974)* per exemple:

In [31]:
pelicula=str(input())
#Godfather: Part II, The (1974)
df_jaccard[pelicula].nlargest(14)[1:]

Godfather: Part II, The (1974)


title
Casino (1995)                      0.500000
Donnie Brasco (1997)               0.500000
Godfather, The (1972)              0.500000
Goodfellas (1990)                  0.500000
Married to the Mob (1988)          0.500000
Miller's Crossing (1990)           0.500000
My Blue Heaven (1990)              0.500000
Scarface (1983)                    0.500000
Carlito's Way (1993)               0.250000
Godfather: Part III, The (1990)    0.166667
Insomnia (2002)                    0.142857
Pulp Fiction (1994)                0.005747
(500) Days of Summer (2009)        0.000000
Name: Godfather: Part II, The (1974), dtype: float64

Totes les pel·lícules tenen a veure amb la mafia, per tant funciona bé.

- Procedim a calcular la similitud de Jaccard per a multiconjunts (bags)

Fem una copia del DataFrame del primer apartat:

In [10]:
bag=pivot.copy()

bag=bag.astype(int)
print(bag.shape)
bag.head()

(1572, 1589)


tag,"""artsy""",06 Oscar Nominated Best Movie - Animation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001-like,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer (2009),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...And Justice for All (1979),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10 Cloverfield Lane (2016),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10 Things I Hate About You (1999),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101 Dalmatians (1996),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Calculem la **bag of words** de cada pel·lícula i l'afegim en una nova columna. Per fer-ho, apliquem minúscula a tots els tags i eliminem els espais dels tags on n'hi hagi, així doncs, per exemple el tag **Al Pacino** quedarà representat a la bag of words com **alpacino**:

In [23]:
l=[]
for index, row in bag.iterrows():
  words = ''
  for col in bag.columns:
    if row[col] > 0:
      for i in range(1,row[col]+1):
        string=col.lower().replace(' ','')#apliquem lowercase al tag i eliminem espais en cas que n'hi hagi
        words += ' '+string+' '
  l.append(words)
print(len(l))

1572


In [24]:
bag['Bag_of_words']=l

Comprovem que a la **bag of words** de les pel·lícules hi figuren tots els tags:

In [32]:
bag.loc[pelicula]['Bag_of_words']

' alpacino  mafia  mafia '

In [33]:
pivot.loc[pelicula][pivot.loc[pelicula]>0].sort_values(ascending=False)

tag
Mafia        2.0
Al Pacino    1.0
Name: Godfather: Part II, The (1974), dtype: float64

Eliminem espais a davant i a darrera de l'string així com espais múltiples entre tags i creem la bag of words com una llista on cada posició es un tag:

In [34]:
bag['Bag_of_words'] = bag['Bag_of_words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')

In [35]:
bag.loc[pelicula]['Bag_of_words'].split()

['alpacino', 'mafia', 'mafia']

Definim una funció per calcular la distància de Jaccard multiset.

In [ ]:
#Multiset jaccard distance
def jaccard_multiset(A,B):
  num=0 #Numerador de la fracció
  den=0 #Denominador de la fracció
  for i in set(A):
    den+=A.count(i)+B.count(i)
    if A.count(i)<=B.count(i):
      num+=A.count(i)
    else:
      num+=B.count(i)
  jaccard_multi=num/den
  return jaccard_multi

Afegim una columna amb els títols de les pel·lícules:

In [36]:
bag['title']=bag.index

Calculem la similitud de jaccard entre la *bag of words* de la pel·lícula especificada i la resta de pel·lícules, guardem la informació en un diccionari i imprimim les **N** pel·lícules amb similitud més alta:

In [ ]:
from operator import itemgetter
N =14
d={}
for index, row in bag.iterrows():
  j=jaccard_multiset(bag.loc[pelicula]['Bag_of_words'].split(),row['Bag_of_words'].split())
  d[row['title']]=j
dict(sorted(d.items(), key = itemgetter(1), reverse = True)[1:N])  

{"Carlito's Way (1993)": 0.4,
 'Godfather: Part III, The (1990)': 0.4,
 'American Gangster (2007)': 0.25,
 'Casino (1995)': 0.25,
 'Donnie Brasco (1997)': 0.25,
 'Godfather, The (1972)': 0.25,
 'Goodfellas (1990)': 0.25,
 'Insomnia (2002)': 0.25,
 'Married to the Mob (1988)': 0.25,
 "Miller's Crossing (1990)": 0.25,
 'My Blue Heaven (1990)': 0.25,
 'Pulp Fiction (1994)': 0.25,
 'Scarface (1983)': 0.25}

Veiem com les pel·lícules amb més similitud són molt similars a seleccionades en la similitud per conjunts, l'ordre està alterat degut a que es té en compte el número de vegades que apareix cada tag a la bag of words, per exemple apareix *American Gangster* a la posició 3, mentre que en l'anterior ranking no apareixia. Hi ha un número major de pel·lícules amb similitud > 0. En la meva opinió aquestes recomanacions són millors que les anteriors. 

- Procedim a calcular la similitud del cosinus.

Creem un vector de representació de la **bag of words**:

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(bag['Bag_of_words'])
count_matrix

<1572x1505 sparse matrix of type '<class 'numpy.int64'>'
	with 3689 stored elements in Compressed Sparse Row format>

Calculem la similitud del cosinus entre tots els parells de pel·lícules:

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


Creem un dataframe amb la **similitud del cosinus** entre tots els parells de pel·lícules:

In [ ]:
df_cosinus=pd.DataFrame(cosine_sim, index=bag.index,columns=bag.index)

Imprimim les pel·lícules més similars:

In [ ]:
df_cosinus[pelicula].nlargest(14)[1:]

title
Casino (1995)                      0.894427
Donnie Brasco (1997)               0.894427
Godfather, The (1972)              0.894427
Goodfellas (1990)                  0.894427
Married to the Mob (1988)          0.894427
Miller's Crossing (1990)           0.894427
My Blue Heaven (1990)              0.894427
Carlito's Way (1993)               0.774597
Godfather: Part III, The (1990)    0.600000
American Gangster (2007)           0.447214
Scarface (1983)                    0.447214
Insomnia (2002)                    0.141421
Pulp Fiction (1994)                0.059235
Name: Godfather: Part II, The (1974), dtype: float64

El ranking és molt similar al calculat amb la **similitud de Jaccard per conjunts**, en aquest cas entra al Top 10 *American Gangster (2007)* i desapareix *Scarface (1983)*. 

Finalment ho condensem tot en una funció:

In [ ]:
pelicula=str(input())
#Godfather: Part II, The (1974)
def recomanador(pelicula):
  print('Recomanacions similars a la pel·lícula:',pelicula)
  print('--------------------------------------------------------')
  print('Recomanacions segons similitud de Jaccard per a conjunts')
  print('--------------------------------------------------------')
  print(df_jaccard[pelicula].nlargest(14)[1:])
  print('--------------------------------------------------------')
  print('Recomanacions segons similitud de Jaccard per a multiconjunts (bags)')
  print('--------------------------------------------------------')
  N =14
  d={}
  for index, row in bag.iterrows():
    j=jaccard_multiset(bag.loc[pelicula]['Bag_of_words'].split(),row['Bag_of_words'].split())
    d[row['title']]=j
  for i in dict(sorted(d.items(), key = itemgetter(1), reverse = True)[1:N]):
    print(i)
  print('--------------------------------------------------------')  
  print('Recomanacions segons similitud del cosinus')
  print('--------------------------------------------------------')  
  print(df_cosinus[pelicula].nlargest(14)[1:])
  #print(list(dict(sorted(d.items(), key = itemgetter(1), reverse = True)[1:N])))

recomanador(pelicula)

Godfather: Part II, The (1974)
Recomanacions similars a la pel·lícula: Godfather: Part II, The (1974)
--------------------------------------------------------
Recomanacions segons similitud de Jaccard per a conjunts
--------------------------------------------------------
title
Casino (1995)                      0.500000
Donnie Brasco (1997)               0.500000
Godfather, The (1972)              0.500000
Goodfellas (1990)                  0.500000
Married to the Mob (1988)          0.500000
Miller's Crossing (1990)           0.500000
My Blue Heaven (1990)              0.500000
Scarface (1983)                    0.500000
Carlito's Way (1993)               0.250000
Godfather: Part III, The (1990)    0.166667
Insomnia (2002)                    0.142857
Pulp Fiction (1994)                0.005747
(500) Days of Summer (2009)        0.000000
Name: Godfather: Part II, The (1974), dtype: float64
--------------------------------------------------------
Recomanacions segons similitud de Jacca

#c) Permeti demanar recomanacions per un usuari determinat basades en el seu perfil. Us heu de pensar vosaltres la manera de construir el perfil.

## He fet servir 2 mètodes per construir el perfil d'usuari i fer-ne recomanacions, aquest és el primer:

Aquest *approach* es basa en que l'usuari introdueixi els títols de les pel·lícules que li han agradat, a partir d'aquests títols es construeix una *bag of words* amb els tags de les pel·lícules que han agradat a l'usuari, a partir d'aquesta *bag of words* es calcula la similitud amb els tags de totes les pel·lícules que l'usuari no ha vist mitjançant el mètode de *similitud del cosinus* i es recomanen les més similars:

In [ ]:
for i in bag['title']:
  print(i)

L'usuari introdueix els títols de les pel·lícules que li han agradat i aquests es guarden en una llista **perfil**:

In [ ]:
likes = input("Introdueix pel·lícules que t'agradin separades per coma:")
#West Side Story (1961),Titanic (1997)
perfil = likes.split(",")
if all(x in bag['title'] for x in perfil) == False:
      perfil=None
      print('La pel·lícula no figura a la base de dades.')

Introdueix pel·lícules que t'agradin separades per coma:West Side Story (1961),Titanic (1997)


In [ ]:
perfil

['West Side Story (1961)', 'Titanic (1997)']

In [ ]:
usuari = bag.reindex(perfil)
usuari['Bag_of_words']

title
West Side Story (1961)                gangs
Titanic (1997)            romance shipwreck
Name: Bag_of_words, dtype: object

Creem una bag of words amb els tags de les pel·lícules que l'usuari ha introduït:

In [ ]:
bag_usuari=''

for i in perfil:
  bag_usuari+=usuari.loc[i]['Bag_of_words']+' '

print(bag_usuari)


gangs romance shipwreck 


Creem un DataFrame amb les pel·lícules i *bag of words* que no figuren a la llista introduïda per l'usuari. Afegim el perfil d'usuari al DataFrame com si fos una pel·lícula més.

In [ ]:
non_user_movies=bag.drop(perfil,axis=0)
non_user_movies=non_user_movies[['Bag_of_words']]
non_user_movies.loc['perfil']=bag_usuari

In [ ]:
non_user_movies['Bag_of_words']

title
(500) Days of Summer (2009)          funny zooeydeschanel artistic humorous inspiri...
...And Justice for All (1979)                                                  lawyers
10 Cloverfield Lane (2016)                                             creepy suspense
10 Things I Hate About You (1999)                                    shakespearesortof
101 Dalmatians (1996)                                                      dogs remake
                                                           ...                        
Zombieland (2009)                    billmurray emmastone jesseeisenberg woodyharre...
Zoolander (2001)                     davidbowie willferrell benstiller comedy goofy...
Zulu (1964)                                                                     africa
eXistenZ (1999)                                                         virtualreality
perfil                                                        gangs romance shipwreck 
Name: Bag_of_words, Length: 1571, dty

Vectoritzem la columna *bag of words* i apliquem la similitud del cosinus entre totes les files del DataFrame:

In [ ]:
count = CountVectorizer()
count_matrix_pusuari = count.fit_transform(non_user_movies['Bag_of_words'])
count_matrix_pusuari

<1571x1505 sparse matrix of type '<class 'numpy.int64'>'
	with 3689 stored elements in Compressed Sparse Row format>

In [ ]:
cosine_sim_pusuari = cosine_similarity(count_matrix_pusuari, count_matrix_pusuari)
print(cosine_sim_pusuari)

[[1.         0.         0.         ... 0.         0.         0.20412415]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.20412415 0.         0.         ... 0.         0.         1.        ]]


Creem un DataFrame amb resultats i imprimim les pel·lícules més similars al **perfil**:

In [ ]:
df_cosinus_pusuari=pd.DataFrame(cosine_sim_pusuari, index=non_user_movies.index,columns=non_user_movies.index)

In [ ]:
df_cosinus_pusuari['perfil'].nlargest(6)[1:6]

title
Better Luck Tomorrow (2002)     0.577350
Notebook, The (2004)            0.333333
Town, The (2010)                0.288675
Stranger than Fiction (2006)    0.235702
(500) Days of Summer (2009)     0.204124
Name: perfil, dtype: float64

## Segon mètode per construir el perfil d'usuari:

Aquest segon *approach* es basa en construir un perfil d'usuari a partir dels tags de les pel·lícules que li agraden i calcular la similitud d'aquest perfil amb totes les pel·lícules mitjançant la similitud de Jaccard per conjunts.

Creem una llista amb tots els tags diferents que s'han donat a les pel·lícules, anomenada **etiquetes**:

In [ ]:
etiquetes=list(pivot.columns)
len(set(etiquetes))

1589

In [ ]:
etiquetes

Definim un *perfil d'usuari* com una llista d'etiquetes, on hi ha les etiquetes característiques de les pel·lícules que agraden a l'usuari:

In [ ]:
perfil = ['New York','New York City']

Fem una copia del DataFrame binari utilitzat per calcular la similitud de Jaccard per conjunts:

In [ ]:
personalized=binary.copy()

Emplenem una llista binària amb un 1 a les columnes del DataFrame que corresponen a l'etiqueta i afegim la llista al dataframe com si fos una pel·lícula més.

In [ ]:
profile=[]
for col in personalized.columns:
  if col in perfil:
    profile.append(1)
  else:
    profile.append(0)

len(profile)

1589

In [ ]:
personalized.loc['perfil'] = profile

Busquem les pel·lícules que tenen més similitud amb les etiquetes especificades per l'usuari segons el métode de Jaccard de similitud per conjunts d'igual manera que hem fet al primer punt de l'apartat anterior i les imprimim:

In [ ]:
jaccard_d=pdist(personalized.values, metric='jaccard')
square_jaccard_d=squareform(jaccard_d)
square_jaccard_s=1-square_jaccard_d
df_j=pd.DataFrame(square_jaccard_s, index=personalized.index,columns=personalized.index)
df_j['perfil'].nlargest(9)[1:]

title
Annie Hall (1977)                  0.500000
Crossing Delancey (1988)           0.500000
Moonstruck (1987)                  0.500000
When Harry Met Sally... (1989)     0.500000
Caveman's Valentine, The (2001)    0.333333
Kids (1995)                        0.250000
Dark Days (2000)                   0.142857
Friends with Benefits (2011)       0.100000
Name: perfil, dtype: float64

Finalment fem un wrap de tot en una sola funció i l'invoquem amb els tags *New York i New York City*

In [ ]:
def pelicules_similars():
  p = input("Introdueix una o vàries etiquetes separades per coma:")
  perfil = p.split(",")
  if all(x in list(etiquetes) for x in perfil) == False:
        print('Etiqueta errònea')
  else:
    personalized=binary.copy()
    profile=[]
    for col in personalized.columns:
      if col in perfil:
        profile.append(1)
      else:
        profile.append(0)
    personalized.loc['perfil'] = profile
    jaccard_d=pdist(personalized.values, metric='jaccard')
    square_jaccard_d=squareform(jaccard_d)
    square_jaccard_s=1-square_jaccard_d
    df_j=pd.DataFrame(square_jaccard_s, index=personalized.index,columns=personalized.index)
    return df_j['perfil'].nlargest(9)[1:]

In [ ]:
#New York,New York City
recomanacions = pelicules_similars()
print(recomanacions)

Introdueix una o vàries etiquetes separades per coma:New York,New York City
title
Annie Hall (1977)                  0.500000
Crossing Delancey (1988)           0.500000
Moonstruck (1987)                  0.500000
When Harry Met Sally... (1989)     0.500000
Caveman's Valentine, The (2001)    0.333333
Kids (1995)                        0.250000
Dark Days (2000)                   0.142857
Friends with Benefits (2011)       0.100000
Name: perfil, dtype: float64


#d/ Opcionalment, implementeu un recomanador basat en el mètode Naive Bayes explicat a classe que, donat un usuari, li recomani les pel·lícules que es més probable que li agradin donats els tags de les pel·lícules que li han agradat (les ha qualificat amb bona puntuació) i les que no (amb puntuació baixa)


In [11]:
import numpy as np

In [12]:
ratings = pd.read_csv("/content/drive/MyDrive/Advanced ML/ratings.csv")
print('ratings shape:',ratings.shape)
print(ratings.columns)
ratings.head()

ratings shape: (100836, 4)
Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Afegim la columna *titles* al DataFrame *ratings*

In [13]:
ratings = pd.merge(ratings[['userId','movieId','rating']],movies[['movieId','title']],on=['movieId'],how='left')
ratings.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Grumpier Old Men (1995)
2,1,6,4.0,Heat (1995)
3,1,47,5.0,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,"Usual Suspects, The (1995)"


Creem un DataFrame on els **indexs són les Id dels usuaris**, les **columnes són els títols** i els **valors, el rating** que l'usuari ha donat a la pel·lícula.

In [16]:
users_title=ratings.pivot_table(values='rating',index='userId',columns='title')
print(users_title.shape)
users_title.head()

(610, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Donat un usuari (agafem el primer a mode d'exemple), iterem entre totes les pel·lícules i guardem les que ha puntuat per sobre del 2.5 (equivalent a 5 sobre 10) en una llista de pel·lícules que li han agradat, també guardem les puntuades per sota del 2.5 en un altre llista de pel·lícules que no li han agradat.

In [52]:
Id=1
like_films=[]
dislike_films=[]
for col in users_title.loc[Id][users_title.loc[Id].notnull()].index:
  if users_title.loc[Id][col]>=2.5:
    like_films.append(col)
  else:
    dislike_films.append(col)

print('Pel·lícules que li han agradat: ',len(like_films))
print('Pel·lícules que no li han agradat: ',len(dislike_films))

Pel·lícules que li han agradat:  226
Pel·lícules que no li han agradat:  6


Com que no totes les pel·lícules puntuades per l'usuari tenen tags, filtrem les llistes per a que només hi apareixin pel·lícules amb tags:

In [37]:
from itertools import compress

mask=[x in bag['title'] for x in dislike_films]
dislike_films=list(compress(dislike_films, mask))
print('Pel·lícules amb tags que no li han agradat: ',len(dislike_films))

mask=[x in bag['title'] for x in like_films]
like_films=list(compress(like_films, mask))
print('Pel·lícules amb tags que li han agradat: ',len(like_films))

Pel·lícules amb tags que no li han agradat:  2
Pel·lícules amb tags que li han agradat:  112


Calculem la bag of words de les pel·lícules que no li han agradat:

In [38]:
bag_dislike=''

for i in dislike_films:
  bag_dislike+=bag.loc[i]['Bag_of_words']+' '

print(bag_dislike)

alfredhitchcock alfredhitchcock normanbates blackandwhite imdbtop250 psychology remade suspenseful tense europe judelaw basedonabook creepy disturbing murder murder obsession psychology secrets serialkiller suspense 


D'igual manera, calculem la bag of words de les pel·lícules que li han agradat:

In [39]:
bag_like=''

for i in like_films:
  bag_like+=bag.loc[i]['Bag_of_words']+' '

print(bag_like)

swashbuckler disney aliens edwardnorton emotional politics racism thought-provoking vietnam timetravel superhero superhero children coenbrothers cultclassic highlyquotable jeffbridges johngoodman juliannemoore nudity(fullfrontal) philipseymourhoffman stevebuscemi blackcomedy bowling classic coenbrothers comedy crime cultfilm darkcomedy deadpan drugs funny greatdialogue greatsoundtrack imdbtop250 kidnapping marijuana off-beatcomedy quirky ransom rug sarcasm satirical violence adamsandler school stoplookingatmeswan darkhumor easygoing silly saturdaynightlive crime off-beatcomedy quirky medieval oscar(bestcinematography) scotland beautifulscenery epic historical inspirational melgibson revenge swordfight rosebud tomclancy cynical generationx hilarious independentfilm quirky witty brainwashing australia americanindians nativeamericans comicbook kevinsmith irreverent jayandsilentbob religion satire disney aliens moviebusiness freaks england kingarthur transplants disney coenbrothers kidnapp

Creem una llista amb el total de pel·lícules puntuades amb tags:

In [40]:
rated_films=like_films+dislike_films
print(len(rated_films))

114


Fem un subset del DataFrame **bag** amb les pel·lícules que **no ha puntuat** l'usuari:

In [41]:
non_rated_movies=bag.drop(rated_films,axis=0)
len(non_rated_movies)

1458

Calculem la probabilitat **P(tag|SPAM)** de tots els tags de la **bag_dislike**, utilitzem un factor suavitzador de Laplace **k=1** i els afegim a un diccionari en format {'tag':prob}

In [42]:
dict_spamicity = {}
total_spam = len(bag_dislike.split())
k=1
for t in set(bag_dislike.split()):
#  print(f"Number of tags {t} in spam films: {bag_dislike.split().count(t)}")
  spamicity = (bag_dislike.split().count(t)+k)/(total_spam+k*len(set(bag_dislike.split()+bag_like.split()))) #p(tag|SPAM)
#  print(f"Spamicity of the word '{t}': {spamicity} \n")
  dict_spamicity[t] = spamicity

print(dict_spamicity)

{'murder': 0.007177033492822967, 'normanbates': 0.004784688995215311, 'europe': 0.004784688995215311, 'serialkiller': 0.004784688995215311, 'creepy': 0.004784688995215311, 'psychology': 0.007177033492822967, 'basedonabook': 0.004784688995215311, 'secrets': 0.004784688995215311, 'suspense': 0.004784688995215311, 'remade': 0.004784688995215311, 'imdbtop250': 0.004784688995215311, 'alfredhitchcock': 0.007177033492822967, 'disturbing': 0.004784688995215311, 'obsession': 0.004784688995215311, 'blackandwhite': 0.004784688995215311, 'tense': 0.004784688995215311, 'judelaw': 0.004784688995215311, 'suspenseful': 0.004784688995215311}


Calculem la probabilitat **P(tag|HAM)** de tots els tags de la **bag_like**, utilitzem un factor suavitzador de Laplace **k=1** i els afegim a un diccionari en format {'tag':prob}

In [43]:
dict_hamicity = {}
total_ham = len(bag_like.split())
for t in set(bag_like.split()):         
#  print(f"Number of tags {t} in ham films: {bag_like.split().count(t)}") #contador del tag
  Hamicity = (bag_like.split().count(t)+k)/(total_ham+k*len(set(bag_dislike.split()+bag_like.split()))) #p(tag|HAM)
#  print(f"Hamicity of the word '{t}': {Hamicity} ")
  dict_hamicity[t] = Hamicity

print(dict_hamicity)

{'adamsandler': 0.0020639834881320948, 'transplants': 0.0020639834881320948, 'rome': 0.0020639834881320948, 'police': 0.0030959752321981426, 'starwars': 0.0030959752321981426, 'shakespeare': 0.0020639834881320948, 'consumerism': 0.0020639834881320948, 'dialogue': 0.0030959752321981426, 'brainwashing': 0.0020639834881320948, 'nudity(fullfrontal)': 0.0020639834881320948, 'veryfunny': 0.0020639834881320948, 'pulp': 0.0020639834881320948, 'beautifulscenery': 0.0020639834881320948, 'dinosaur': 0.0020639834881320948, 'complicated': 0.0020639834881320948, 'tarantino': 0.0041279669762641896, 'vietnam': 0.005159958720330237, 'r:strongbloodyviolence': 0.0020639834881320948, 'clever': 0.0020639834881320948, 'moldy': 0.0020639834881320948, 'coolstyle': 0.0020639834881320948, 'southpark': 0.0020639834881320948, 'entertaining': 0.0020639834881320948, 'interwovenstorylines': 0.0020639834881320948, 'stevenspielberg': 0.0020639834881320948, 'court': 0.0020639834881320948, 'classic': 0.00825593395252837

Calculem la **p(SPAM)** i la **p(HAM)** de les pel·lícules amb **factor de suavització k=1**

In [44]:
prob_spam=(len(dislike_films)+k)/(len(rated_films)+k*2) #p(SPAM)
print(prob_spam)

prob_ham=(len(like_films)+k)/(len(rated_films)+k*2) #p(HAM)
print(prob_ham)

0.02586206896551724
0.9741379310344828


Extraiem una pel·lícula del dataset a mode d'exemple:

In [45]:
exemple = non_rated_movies.loc['12 Angry Men (1957)']['Bag_of_words']

Iterem entre els tags de la pel·lícula i enmagatzemem en una llista els que apareixen a qualsevol de les dues *bag of words*, la d'spam o la de ham. No tenim en compte els tags que no apareixen a les bags.

In [46]:
words_ = []
for word in exemple.split():
    if word in set(bag_like.split()):
        print(f"'{word}', ok")
        words_.append(word)
    elif word in set(bag_dislike.split()):
        print(f"'{word}', ok")
        words_.append(word)
    else:
        print(f"'{word}', el tag no apareix a les bags of words")
print(words_)

'motivational', el tag no apareix a les bags of words
'claustrophobic', el tag no apareix a les bags of words
'confrontational', el tag no apareix a les bags of words
'court', ok
'earnest', el tag no apareix a les bags of words
'gooddialogue', ok
'greatscreenplay', el tag no apareix a les bags of words
'gritty', ok
'thought-provoking', ok
['court', 'gooddialogue', 'gritty', 'thought-provoking']


Definim una funció que, donada la llista de tags d'una pel·lícula, calculi la probabilitat d'spam i ham de cada tag **P(SPAM|tag) i P(HAM|tag)** amb un factor suavitzador *k=1* i enmagatzemi totes les probabilitats en una llista. També definim una segona funció que multipliqui les probabilitats de cada llista i retorni **P(SPAM|pel·lícula) i P(HAM|pel·lícula)**. Finalment classifiquem la pel·lícula com a spam si la probabilitat d'spam és superior a PHI vegades la probabilitat de ham **P(SPAM|pel·lícula)>PHI*P(HAM|pel·lícula)**,
adoptem **PHI=1.5**

In [47]:
def mult(list_) :        # funcio multiplicació probabilitats
    total_prob = 1
    for i in list_: 
         total_prob = total_prob * i  
    return total_prob
    
def Bayes(film):
  k=1
  PHI=1.5 
  probs_spam = []
  probs_ham = []
  for word in film:#iterem entre tots els tags
    Pr_S = prob_spam #p(SPAM)
    try:
      pr_WS = dict_spamicity[word] #P(tag|SPAM)
    except KeyError:
      pr_WS = (bag_dislike.split().count(word)+k)/(total_spam+k*len(set(bag_dislike.split()+bag_like.split()))) #P(tag|SPAM)

    Pr_H = prob_ham #p(HAM)
    try:    
      pr_WH = dict_hamicity[word] #P(tag|HAM) 
    except KeyError:
      pr_WH = (bag_like.split().count(word)+k)/(total_ham+k*len(set(bag_dislike.split()+bag_like.split()))) #P(tag|HAM)

    prob_word_is_spam_BAYES = (pr_WS*Pr_S)/((pr_WS*Pr_S)+(pr_WH*Pr_H)) #P(SPAM|tag)=(P(tag|SPAM)*p(SPAM))/(P(tag|SPAM)*p(SPAM)+P(tag|HAM)*p(HAM))
    prob_word_is_ham_BAYES = (pr_WH*Pr_S)/((pr_WS*Pr_S)+(pr_WH*Pr_H)) #P(HAM|tag)=(P(tag|HAM)*p(HAM))/(P(tag|SPAM)*p(SPAM)+P(tag|HAM)*p(HAM))
#    print(prob_word_is_spam_BAYES)
    probs_spam.append(prob_word_is_spam_BAYES)
    probs_ham.append(prob_word_is_ham_BAYES)
  final_classification_spam = mult(probs_spam)
  final_classification_ham = mult(probs_ham)
  print('P(SPAM|pel·lícula): ',final_classification_spam)
  print('P(HAM|pel·lícula): ',final_classification_ham)
  if final_classification_spam > PHI*final_classification_ham:
    return False
  else:
    return True

Apliquem la funció per a que ens classifiqui la pel·lícula d'exemple com a recomanada o spam:

In [48]:
recommended_films=[]
if Bayes(words_):
  recommended_films.append('12 Angry Men (1957)')
print('Llistat de pel·lícules recomanades')
print('--------------------------------------------------------')
for i in recommended_films:
  print(i)

P(SPAM|pel·lícula):  1.8190801256102204e-07
P(HAM|pel·lícula):  4.5351780739635444e-07
Llistat de pel·lícules recomanades
--------------------------------------------------------
12 Angry Men (1957)


Un cop explicat tot el procediment, sintetitzem tots els passos realitzats en pocs en blocs i iterem entre totes les pel·lícules que no han estat puntuades per l'usuari, aplicant el filtre d'spam pel mètode de **Naive Bayes** i imprimim els títols recomanats:

In [49]:
Id=1
like_films=[]
dislike_films=[]
for col in users_title.loc[Id][users_title.loc[Id].notnull()].index:
  if users_title.loc[Id][col]>=2.5:
    like_films.append(col)
  else:
    dislike_films.append(col)

In [50]:
from itertools import compress

mask=[x in bag['title'] for x in dislike_films]
dislike_films=list(compress(dislike_films, mask))
print("Pel·lícules que NO agraden a l'usuari i contenen tags:", len(dislike_films))

mask=[x in bag['title'] for x in like_films]
like_films=list(compress(like_films, mask))
print("Pel·lícules que agraden a l'usuari i contenen tags:", len(like_films))

bag_dislike=''

for i in dislike_films:
  bag_dislike+=bag.loc[i]['Bag_of_words']+' ' #bag of words de tags de les pel·lícules que no li han agradat

bag_like=''

for i in like_films:
  bag_like+=bag.loc[i]['Bag_of_words']+' ' #bag of words de tags de les pel·lícules que li han agradat

rated_films=like_films+dislike_films #pel·lícules puntuades per l'usuari i amb tags
non_rated_movies=bag.drop(rated_films,axis=0) #DataFrame amb pel·lícules no puntuades i amb tags

vocab_unique_words_spam = set(bag_dislike.split()) #Vocabulary tags spam

dict_spamicity = {}
total_spam = len(bag_dislike.split())
k=1
for t in vocab_unique_words_spam:
  spamicity = (bag_dislike.split().count(t)+k)/(total_spam+k*len(set(bag_dislike.split()+bag_like.split()))) #p(tag|SPAM) 
  dict_spamicity[t] = spamicity

vocab_unique_words_ham = set(bag_like.split()) #Vocabulary tags ham

dict_hamicity = {}
total_ham = len(bag_like.split())
for t in vocab_unique_words_ham:
            
  Hamicity = (bag_like.split().count(t)+k)/(total_ham+k*len(set(bag_dislike.split()+bag_like.split()))) #p(tag|HAM)
  dict_hamicity[t] = Hamicity

prob_spam=(len(dislike_films)+k)/(len(rated_films)+k*2) #p(SPAM)

prob_ham=(len(like_films)+k)/(len(rated_films)+k*2) #p(HAM)

def mult(list_) :#Funció per multiplicar els elements d'una llista        
    total_prob = 1
    for i in list_: 
         total_prob = total_prob * i  
    return total_prob
    
def Bayes(film):#Filtre d'spam donat la bag of words d'una pel·lícula
  k=1
  PHI=1.5 
  probs_spam = []
  probs_ham = []
  for word in film:
    Pr_S = prob_spam #p(SPAM)
    try:
      pr_WS = dict_spamicity[word] #P(tag|SPAM)
    except KeyError:#si la paraula no és al diccionari
      pr_WS = (bag_dislike.split().count(word)+k)/(total_spam+k*len(set(bag_dislike.split()+bag_like.split()))) #P(tag|SPAM)

    Pr_H = prob_ham #p(HAM)
    try:    
      pr_WH = dict_hamicity[word] #P(tag|HAM) 
    except KeyError:#si la paraula no és al diccionari
      pr_WH = (bag_like.split().count(word)+k)/(total_ham+k*len(set(bag_dislike.split()+bag_like.split()))) #P(tag|HAM)

    prob_word_is_spam_BAYES = (pr_WS*Pr_S)/((pr_WS*Pr_S)+(pr_WH*Pr_H))#P(SPAM|tag)=(P(tag|SPAM)*p(SPAM))/(P(tag|SPAM)*p(SPAM)+P(tag|HAM)*p(HAM))
    prob_word_is_ham_BAYES = (pr_WH*Pr_S)/((pr_WS*Pr_S)+(pr_WH*Pr_H))#P(HAM|tag)=(P(tag|HAM)*p(HAM))/(P(tag|SPAM)*p(SPAM)+P(tag|HAM)*p(HAM))

    probs_spam.append(prob_word_is_spam_BAYES)
    probs_ham.append(prob_word_is_ham_BAYES)
  final_classification_spam = mult(probs_spam)
  final_classification_ham = mult(probs_ham)

  if final_classification_spam > PHI*final_classification_ham:
    return False
  else:
    return True

Pel·lícules que NO agraden a l'usuari i contenen tags: 2
Pel·lícules que agraden a l'usuari i contenen tags: 112


Iterem totes les pel·lícules del DataFrame i apliquem tots els passos a totes les pel·lícules, emmagatzemem les que passen el filtre d'spam a la llista **recommended_films** i les imprimim:

In [54]:
recommended_films=[]
for film in non_rated_movies.index:
  movie = non_rated_movies.loc[film]['Bag_of_words']
  words_ = []
  for word in movie.split():
    if word in set(bag_like.split()):
#        print(f"'{word}', ok")
      words_.append(word)
    elif word in set(bag_dislike.split()):
#        print(f"'{word}', ok")
      words_.append(word)
  if Bayes(words_):#Passem el filtre d'Spam a la bag of words de la pel·lícula
    recommended_films.append(film)
print('Llistat de pel·lícules recomanades')
print('--------------------------------------------------------')
for i in recommended_films:
  print(i)

Llistat de pel·lícules recomanades
--------------------------------------------------------
(500) Days of Summer (2009)
...And Justice for All (1979)
10 Things I Hate About You (1999)
101 Dalmatians (1996)
101 Dalmatians (One Hundred and One Dalmatians) (1961)
11'09"01 - September 11 (2002)
12 Angry Men (1957)
127 Hours (2010)
13 Going on 30 (2004)
2001: A Space Odyssey (1968)
21 Grams (2003)
25th Hour (2002)
28 Days Later (2002)
39 Steps, The (1935)
3:10 to Yuma (2007)
40-Year-Old Virgin, The (2005)
400 Blows, The (Les quatre cents coups) (1959)
42 Up (1998)
84 Charing Cross Road (1987)
8MM (1999)
A Million Ways to Die in the West (2014)
A Pigeon Sat on a Branch Reflecting on Existence (2014)
A Story of Children and Film (2013)
A.I. Artificial Intelligence (2001)
About a Boy (2002)
Accused, The (1988)
Adam's Rib (1949)
Addams Family Values (1993)
Addams Family, The (1991)
African Queen, The (1951)
After the Thin Man (1936)
Age of Innocence, The (1993)
Air Force One (1997)
Airheads (19

In [70]:
spam_films=len(non_rated_movies.index)-len(recommended_films)
print('Total de pel·lícules SPAM: ',spam_films)
print('Total de pel·lícules recomanades: ',len(recommended_films))

Total de pel·lícules SPAM:  46
Total de pel·lícules recomanades:  1412
